In [7]:
# To handle event loop error by nesting it onto the runtime loop
import nest_asyncio

nest_asyncio.apply()

initializng logger and imports

In [2]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SQLDatabase
from llama_index.readers.wikipedia import WikipediaReader

Creating a test sql database using sqllite

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    inspect,
    select,
    column,
)

In [4]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
api_key=""
llm = Groq(model="llama3-8b-8192",
    api_key=api_key,
    temperature=0.5
)
embediing_model = HuggingFaceEmbedding(model_name='thenlper/gte-base')
Settings.llm = llm
Settings.embed_model=embediing_model
engine = create_engine("sqlite:///mydatabase.db", future=True)
metadata_obj = MetaData()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-base
Load pretrained SentenceTransformer: thenlper/gte-base
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


Creating a table in the Initialized database

In [5]:
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [6]:
metadata_obj.tables.keys()
city_stats_table = metadata_obj.tables['city_stats']
city_stats_table

Table('city_stats', MetaData(), Column('city_name', String(length=16), table=<city_stats>, primary_key=True, nullable=False), Column('population', Integer(), table=<city_stats>), Column('country', String(length=16), table=<city_stats>, nullable=False), schema=None)

Adding data into the database

In [44]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

Loading more city data form wikipidea and adding to db. 50 famous cities of asia

In [55]:
# few names causing issue therefore not used
famous_asian_cities = [
    "Tokyo",
    "Beijing",
    "Shanghai",
    "Seoul",
    "Mumbai",
    "Delhi",
    "Bangkok",
    "Singapore",
    # "Hong Kong",
    "Jakarta",
    "Kuala Lumpur",
    # "Manila",
    "Hanoi",
    "Ho Chi Minh City",
    "Dubai",
    "Istanbul",
    "Jerusalem",
    "Kyoto",
    "Osaka",
    # "Chengdu",
    "Taipei",
    # "Xian",
    "Kathmandu",
    "Colombo",
    "Dhaka",
    "Yangon",
    "Ulaanbaatar",
    "Riyadh",
    "Tehran",
    "Baghdad",
    "Beirut",
    # "Amman",
    "Baku",
    "Tashkent",
    # "Almaty",
    "Phnom Penh",
    "Vientiane",
    # "Male",
    "Thimphu",
    "Bandar Seri Begawan",
    # "Macau",
    "George Town",
    "Jeju City",
    # "Busan",
    "Gyeongju",
    "Samarkand",
    "Bukhara"
]
wiki_docs = WikipediaReader().load_data(pages=famous_asian_cities)

Creating a indexing object for table schemas

In [7]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex
sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = []
for table_name in metadata_obj.tables.keys():
    table_schema_objs.append(SQLTableSchema(table_name=table_name))
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


In [9]:

from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)

def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

FnComponent(partial_dict={}, fn=<function get_table_context_str at 0x71d156d31750>, async_fn=None, output_key='output')

In [10]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [11]:

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [12]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [16]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [20]:
response = qp.run(
    query="Which countries are each city from??"
)
print(str(response))
# city_stats_table.columns.keys()

> Running module input with input: 
query: Which countries are each city from??

> Running module table_retriever with input: 
input: Which countries are each city from??



Batches: 100%|██████████| 1/1 [00:00<00:00,  2.68it/s]


> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='city_stats', context_str=None)]

> Running module text2sql_prompt with input: 
query_str: Which countries are each city from??
schema: Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)), and foreign keys: .

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Running module sql_output_parser with input: 
response: assistant: Question: Which countries are each city from??
SQLQuery: SELECT city_stats.country, city_stats.city_name FROM city_stats;
SQLResult: coun